## NEAREST NEIGHBOUR INTERPOLATION

In [11]:
import sys
import cv2
import numpy

def bound(x, y, width, height):
	if x < 0:
		x = 0
	if y < 0:
		y = 0
	if x >= width:
		x = width - 1
	if y >= height:
		y = height - 1
	return x, y

def interpolate_nn(org_img, new_height, new_width):
	height, width, channels = org_img.shape

	height_ratio = height/new_height
	width_ratio = width/new_width
	scaled_img = numpy.zeros((new_height, new_width, channels), org_img.dtype)

	for y_new in range(new_height):
		for x_new in range(new_width):
			y_old = round(y_new * height_ratio)
			x_old = round(x_new * width_ratio)
			#x_old, y_old = bound(x_old, y_old, width, height)

			scaled_img[y_new, x_new] = org_img[y_old, x_old]

	return scaled_img

def main():
	img_path = "img/lion.jpg"
	img = cv2.imread(img_path)

	new_image = interpolate_nn(img, 150, 200)

	cv2.imwrite("output_NN.jpg", new_image)

if __name__ == '__main__':
	main()

## BILINEAR INTERPOLATION

In [4]:
import cv2
import numpy as np
import math

def bound(x, y, height, width):
	if x < 0:
		x = 0
	if y < 0:
		y = 0
	if x >= width:
		x = width - 1
	if y >= height:
		y = height - 1
	return x, y

def interpolate_bilinear(src, new_height, new_width):
    old_height, old_width, channels = src.shape
    
    h_ratio = old_height/new_height 
    w_ratio = old_width/new_width
    
    scaled = np.zeros((new_height,new_width,channels),src.dtype)
        
    for y_new in range(new_height):
        for x_new in range(new_width):
            x_old = x_new * w_ratio
            y_old = y_new * h_ratio
            
            neigh1_x = math.floor(x_old)
            neigh1_y = math.floor(y_old)
            
            neigh1_x,neigh1_y = bound(neigh1_x , neigh1_y, old_height,old_width)
            neigh2_x,neigh2_y = bound(neigh1_x + 1,neigh1_y,old_height,old_width)
            neigh3_x,neigh3_y = bound(neigh1_x ,neigh1_y + 1,old_height,old_width)
            neigh4_x,neigh4_y = bound(neigh1_x + 1,neigh1_y + 1,old_height,old_width)
            
            neigh1_x,neigh1_y = bound(neigh1_x , neigh1_y, old_height,old_width)
            
            delta_x = x_old - neigh1_x
            delta_y = y_old - neigh1_y
            
            scaled[y_new, x_new] = \
            (delta_x)*(delta_y)*src[neigh1_y,neigh1_x] \
            + (1 - delta_x)*(delta_y)*src[neigh2_y,neigh2_x] \
            + (delta_x)*(1 - delta_y)*src[neigh3_y,neigh3_x] \
            + (1 - delta_x)*(1 - delta_y)*src[neigh4_y,neigh4_x]
            
            for c in range(channels):
                scaled[y_new, x_new, c] = round(scaled[y_new, x_new, c])
            
    return scaled

def main():
	img_path = "img/lion.jpg"
	img = cv2.imread(img_path)

	new_image = interpolate_bilinear(img,150,200)

	cv2.imwrite("output_bilinear.jpg", new_image)

if __name__ == '__main__':
	main()   

## BICUBIC INTERPOLATION

In [5]:
import cv2
import numpy as np
import math

def bound(x, y, height, width):
	if x < 0:
		x = 0
	if y < 0:
		y = 0
	if x >= width:
		x = width - 1
	if y >= height:
		y = height - 1
	return x, y

def interpolate_bicubic(src1, new_height, new_width):
    old_height, old_width, channels = src1.shape
    
    SHIFTAMT = 2
    src = cv2.copyMakeBorder(src1,SHIFTAMT,SHIFTAMT,SHIFTAMT,SHIFTAMT,cv2.BORDER_CONSTANT)
    
    
    h_ratio = old_height/new_height 
    w_ratio = old_width/new_width
    
    scaled = np.zeros((new_height,new_width,channels),src.dtype)
        
    for y_new in range(new_height):
        for x_new in range(new_width):
            x_old = x_new * w_ratio + SHIFTAMT
            y_old = y_new * h_ratio + SHIFTAMT
            
            neigh1_x = math.floor(x_old)
            neigh1_y = math.floor(y_old)
            
            delta_x = x_old - neigh1_x
            delta_y = y_old - neigh1_y
            
            X = np.array([[delta_x**i for i in range(3,-1,-1)]]).T
            Y = np.array([[delta_y**i for i in range(3,-1,-1)]])
            
            # relative coordinates's coefficient
            B = np.array([
                [-1, 1, -1, 1], 
                [0, 0, 0, 1],
                [1, 1, 1, 1],
                [8, 4, 2, 1]
            ])
            
            B_inv = np.linalg.inv(B)
                        
            for c in range(channels):
                F = np.array([
                [src[neigh1_y-1,neigh1_x-1,c], src[neigh1_y-1,neigh1_x,c], src[neigh1_y-1,neigh1_x+1,c], src[neigh1_y-1,neigh1_x + 2,c]],
                [src[neigh1_y,neigh1_x-1,c], src[neigh1_y,neigh1_x,c], src[neigh1_y,neigh1_x+1,c], src[neigh1_y,neigh1_x + 2,c]],
                [src[neigh1_y+1,neigh1_x-1,c], src[neigh1_y+1,neigh1_x,c], src[neigh1_y+1,neigh1_x+1,c], src[neigh1_y+1,neigh1_x + 2,c]],
                [src[neigh1_y+2,neigh1_x-1,c], src[neigh1_y+2,neigh1_x,c], src[neigh1_y+2,neigh1_x+1,c], src[neigh1_y+2,neigh1_x + 2,c]]                
                ])
                
                coeff = np.dot(np.dot(B_inv,F),B_inv.T)
#                 print(np.dot(np.dot(Y,coeff),X))
                
                scaled[y_new, x_new, c] = np.dot(np.dot(Y,coeff),X)
                scaled[y_new, x_new, c] = round(scaled[y_new, x_new, c])
            
    return scaled

def main():
	img_path = "img/lion.jpg"
	img = cv2.imread(img_path)

	new_image = interpolate_bicubic(img,150,200)

	cv2.imwrite("output_bicubic.jpg", new_image)

if __name__ == '__main__':
	main()   

### COMPARISION

In [31]:
np.linalg.inv(B_inv)

array([[-0.16666667,  0.5       , -0.5       ,  0.16666667],
       [ 0.5       , -1.        ,  0.5       ,  0.        ],
       [-0.33333333, -0.5       ,  1.        , -0.16666667],
       [ 0.        ,  1.        ,  0.        ,  0.        ]])

In [48]:
cv2.copyMakeBorder(img, 10, 10, 10, 10, cv2.BORDER_CONSTANT)

(1080, 1920, 3)

In [25]:
a = np.array([[2]])

In [26]:
a

array([[2]])

In [27]:
int(a)

2